In [ ]:
import pandas as pd
import glob
import json
import dotted # https://pypi.org/project/dotted-notation/
import re
import matplotlib.pyplot as plt

from pathlib import Path
import seaborn as sns

In [ ]:
import lib.datasciencetoolbelt as dstools
from lib.resultstorage import ResultStorage

In [ ]:
#%matplotlib qt
%matplotlib inline

In [ ]:
dstools.setup({
    "seaborn_context": "talk",
    "savefig": {
        "enable": True,
        "dir": Path("./postprocess_results"),
    }
})
result_storage = ResultStorage(Path("./results"))

In [ ]:
def to_row_dict(output_json):
    r = {}
    for k, v in dotted.get(output_json, "latency_analysis").items():
        assert k[0] == '@'
        k = k[1:] # strip leading @
        assert k not in r
        r[k] = v
    
    r = {
        "numjobs": int(dotted.get(output_json, "fio_config.numjobs")),
        **r
    }
    return r

rows = [to_row_dict(j) for j in result_storage.iter_results("latency_analysis")]


In [ ]:
df = pd.DataFrame.from_dict(rows)
df = df.set_index("numjobs")
df = df.sort_index()
# display(df)
display(df / 1_000_000)
# compute zfs write breakdown

# zfs_write = ASYNC + zfs_log_write_{begin,end} + zil_commit
# zil_commit = ZIL_LWB_OTHER +  zillwb_commit_waiter__timeout  + zillwb_commit_waiter__issue
# zillwb_commit_waiter__issue = ZIL_LWB_ZIOWAIT + pmem_submit_bio
df["zio"] = df.zillwb_commit_waiter__issue - df.pmem_submit_bio
df["zil-lwb-other"] = df.zil_commit - df.zillwb_commit_waiter__timeout - df.zillwb_commit_waiter__issue
df["itx"] = df.zfs_log_write_begin + df.zfs_log_write_finish
df["async"] = df.zfs_write - df.itx - df.zil_commit
# cosmetics
df["zil-lwb-fill-timeout"] = df.zillwb_commit_waiter__timeout
df["zil-lwb-ziowait"] = df.zillwb_commit_waiter__issue - df.pmem_submit_bio

# df["pmem"] = df["pmem_submit_bio"]
# # df["zio"] = df["zio_rewrite"] - df["pmem"]
# df["zil-lwb-fill-timeout"] = df["zillwb_commit_waiter__timeout"]
# df["zil-lwb-ziowait"] = df["zillwb_commit_waiter__issue"]
# df["zil-lwb-other"] = df["zil_commit"] - df["zil-lwb-ziowait"] - df["zil-lwb-fill-timeout"]
# df["itx"] = df["zfs_log_write_begin"] + df["zfs_log_write_finish"]
# df["async"] = df["zfs_write"] - (df["itx"] + df[""])

df["zfs_write__lat_avg"] = df["zfs_write"] / df["zfs_write_count"]
display(df.zfs_write__lat_avg)


# zfs_write_comps = ["async", "itx", "zil", "zio", "pmem"]
zfs_write_comps = [ "async", "itx", "zil-lwb-other", "zil-lwb-fill-timeout", "zil-lwb-ziowait", "pmem_submit_bio"]

# add `unaccounted` component
tmp = df.filter(zfs_write_comps, axis=1)
# display(tmp)
tmp = tmp.transpose().sum()
# display(tmp)
df["unaccounted"] = df["zfs_write"] - tmp
zfs_write_comps += ["unaccounted"]

# relative breakdown (the raw data is aggregate wall-clock time spent in the functions,
# not-so-precise cpaturing interval)

pdata = df.filter(zfs_write_comps, axis=1)
display(pdata / 1_000_000)


In [ ]:
def plot_breakdown(pdata, title, figname, datalabel=None):
    figsize = (14, 10)
    ax = pdata.plot.area(stacked=True, figsize=figsize, sort_columns=True)
    ax.set_title(title, pad=16)
    ax.set_ylabel(datalabel)
    ax.set_xlabel("numjobs")
    
    ax.set_xticks(pdata.index)
    ax.set_xticklabels(pdata.index, rotation=0) # for some reason pandas draws the xlabels rotated
    
    #     ax.set_yticklabels([]) # shows '0' which is confusing unless we have multiple numjobs values
    ax.legend(bbox_to_anchor=(1, 0.5), loc='center left')
    dstools.savefig(f"latency_breakdown_stacked-{figname}")

    # Code for plotting individual bars
    #     ax = pdata.plot.bar(stacked=False, figsize=figsize, sort_columns=True)
    #     ax.set_title(title, pad=16)
    #     ax.set_xlabel(xlabel)
    #     ax.legend(bbox_to_anchor=(1, 0.5), loc='center left')
    #     savefig(f"latency_breakdown_individual_bars-{figname}")
    
title = "zfs_write() relative latency breakdown (sampling overhead!)"
rpdata = pdata.div(df["zfs_write"], axis=0)
display((rpdata * 100).round(1))
plot_breakdown(rpdata, title, "relative")


title = "zfs_write() absolute latency breakdown (sampling overhead!)"
apdata = pdata.div(df["zfs_write"], axis=0).mul(df.zfs_write__lat_avg, axis=0)
display((apdata/1000).round(1))
plot_breakdown(apdata, title, "absolute", datalabel="nano seconds")
    

In [ ]:
# import devdax write latencies from perf analysis
export = '{"schema":{"fields":[{"name":"numjobs","type":"integer"},{"name":"w_lat_mean","type":"number"}],"primaryKey":["numjobs"],"pandas_version":"0.20.0"},"data":[{"numjobs":5,"w_lat_mean":4603.755736},{"numjobs":1,"w_lat_mean":617.493467},{"numjobs":3,"w_lat_mean":2649.305724},{"numjobs":15,"w_lat_mean":40203.617083},{"numjobs":4,"w_lat_mean":3577.32901},{"numjobs":7,"w_lat_mean":6599.553758},{"numjobs":10,"w_lat_mean":16968.396836},{"numjobs":8,"w_lat_mean":8479.026951},{"numjobs":13,"w_lat_mean":29632.536489},{"numjobs":2,"w_lat_mean":1684.654042},{"numjobs":11,"w_lat_mean":22418.170104},{"numjobs":9,"w_lat_mean":12462.04705},{"numjobs":12,"w_lat_mean":26023.822275},{"numjobs":14,"w_lat_mean":34184.87662},{"numjobs":6,"w_lat_mean":5674.396315}]}'
devdax_write_latencies = pd.read_json(export, orient="table")
display(devdax_write_latencies)

In [ ]:
odata = apdata.copy() #! apdata from previous run
odata["devdax_w_lat_mean"] = devdax_write_latencies["w_lat_mean"]
optimal_zfs_write_comps = ["async", "itx", "devdax_w_lat_mean"]
odata.filter(optimal_zfs_write_comps).plot.area(stacked=True)

In [ ]:
opportunities = pd.DataFrame.from_dict({
    "current": apdata.filter(zfs_write_comps).sum(axis=1),
    "optimal": odata.filter(optimal_zfs_write_comps).sum(axis=1),
})
opportunities.plot.line()

In [ ]:
speedup = 1/opportunities.div(opportunities.current, axis=0)
speedup.plot.line(title="Potential Speedup (baseline: current)")